In [1]:
from Bio import SeqIO

def cut_fasta_sequences(input_fasta, output_fasta, ranges_dict):
    """
    Cuts sequences from a FASTA file based on the given ranges and writes them to a new FASTA file.

    Parameters:
    input_fasta (str): Path to the input FASTA file.
    output_fasta (str): Path to the output FASTA file.
    ranges_dict (dict): Dictionary where keys are sequence IDs and values are tuples of (start, end) positions.
                        Positions are 1-based, and both start and end are inclusive.
    """
    # Parse the input FASTA file
    sequences = SeqIO.parse(input_fasta, "fasta")
    
    # Open the output FASTA file
    with open(output_fasta, "w") as output_handle:
        # Iterate over each sequence in the input FASTA file
        for seq_record in sequences:
            seq_id = seq_record.id
            if seq_id in ranges_dict:
                # Get the specified range for the sequence
                start, end = ranges_dict[seq_id]
                # Extract the subsequence within the given range
                cut_seq = seq_record.seq[start-1:end]  # Convert 1-based to 0-based index
                # Create a new SeqRecord with the cut sequence
                new_record = seq_record[start-1:end]
                # Write the new sequence to the output file
                SeqIO.write(new_record, output_handle, "fasta")

# Define the file names and ranges
ranges = {
    "K401": (1, 120),
    "AcSu_575": (1, 120),
    "AcSu2_530": (1, 120),
    "AdPa_408": (1, 120),
    "BleSto_572": (1, 120),
    "DiPu_513": (1, 120),
    "HeAl_513": (1, 120),
    "Kif3_592": (1, 120),
    "Kifa_500": (1, 120),
    "NaGr_392": (1, 120),
    "ThTr_655": (1, 180),
    "TiLa_514": (1, 120)
}

# Example usage:
cut_fasta_sequences("../../data/input_sequences.fasta", "../../data/output_sequences.fasta", ranges)